In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [4]:
api_key="your api key"

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybride-search-langchain-pinecone"
pc=Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct', ## sparse values
        spec=ServerlessSpec(cloud='aws',region="us-east-1"),
    )


c:\Users\abhi\Desktop\udemy GENAI\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


NameError: name 'api_key' is not defined

In [5]:
index=pc.Index(index_name)
index

In [8]:
##vector embedding and sparse matrix
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder=BM25Encoder().default()
bm25_encoder

In [11]:
sentences=[
    "In 2023, I visited paris",
    "In 2022, I visited New york",
    "In 2021, I visited england",
]
bm25_encoder.fit(sentences) #tfidf
bm25_encoder.dump("bm25_values.json")

bm25_encoder=BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<?, ?it/s]


In [12]:
retriver=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [13]:
retriver

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001723976F860>, index=<pinecone.data.index.Index object at 0x0000017259A63F20>)

In [14]:
retriver.add_texts([
    "In 2023, I visited paris",
    "In 2022, I visited New york",
    "In 2021, I visited england",
])

100%|██████████| 1/1 [00:03<00:00,  3.06s/it]


In [16]:
retriver.invoke("what city did I visit first")

[Document(page_content='In 2022, I visited New york'),
 Document(page_content='In 2021, I visited england'),
 Document(page_content='In 2023, I visited paris')]